In [13]:
import pandas as pd
import matplotlib.pyplot as plt 
# import tabulate
%matplotlib inline

In [ ]:
df = pd.read_json("data/silver/1920_2024_geopy.json")
df_country_codes = pd.read_json("data/country_codes.json")

In [ ]:
df[df.Country_location.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_location.csv")
df_country_codes[df_country_codes.name_geopy.apply(lambda x: isinstance(x,str))].to_csv("data/bad_data/unknown_country_codes.csv")

In [ ]:
df_ = df[df.Country_location.apply(lambda x: isinstance(x,dict))]
df_country_codes_ = df_country_codes[df_country_codes.name_geopy.apply(lambda x: isinstance(x,dict))]

In [ ]:
df_country_codes_["name_geopy"][0]

{'place_id': 220100631,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 303427,
 'lat': '33.7680065',
 'lon': '66.2385139',
 'class': 'boundary',
 'type': 'administrative',
 'place_rank': 4,
 'importance': 0.7470274828,
 'addresstype': 'country',
 'name': 'Afghanistan',
 'display_name': 'Afghanistan',
 'boundingbox': ['29.3772926', '38.4910682', '60.5176034', '74.8898620']}

In [ ]:
df_["Country_location"][0]

{'place_id': 372303099,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 1219767831,
 'lat': '54.596391',
 'lon': '-5.9301829',
 'class': 'place',
 'type': 'city',
 'place_rank': 16,
 'importance': 0.6500238354,
 'addresstype': 'city',
 'name': 'Belfast',
 'display_name': 'Belfast, County Antrim, Northern Ireland, United Kingdom',
 'boundingbox': ['54.5306104', '54.6594247', '-6.0454798', '-5.8076879']}

In [ ]:
def unpack_locations(x):
    return ",".join([i for i in x['display_name'].split(",") if i.strip().isnumeric()==False][-2:]).strip()
    # return ",".join(x['display_name'].split(",")[-2:]).strip()

In [ ]:
df_country_codes_["Country_Geopy"] = df_country_codes_.name_geopy.apply(unpack_locations)
df_country_codes_ = df_country_codes_.set_index("Country_Geopy")

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_49440/3496400615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country_codes_["Country_Geopy"] = df_country_codes_.name_geopy.apply(unpack_locations)


In [ ]:
df_["Country_Geopy"] = df_.Country_location.apply(unpack_locations)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_49440/416814173.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Country_Geopy"] = df_.Country_location.apply(unpack_locations)


In [ ]:
df_country_codes_[:3]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy
Country_Geopy,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."


In [ ]:
df_[:3]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Country_Geopy
0,1950,1953-01-05,2139,"Belfast, Northern Ireland",British European Airways,None,Northolt - Belfast,Vickers Viking 610-1B,G-AJDL,262,"{'total': 35, 'passengers': 31, 'crew': 4}","{'total': 27, 'passengers': 24, 'crew': 3}",0.0,Crashed into approach lights while attempting ...,35,27,British European Airways,"{'place_id': 372303099, 'licence': 'Data © Ope...","Northern Ireland, United Kingdom"
1,1950,1953-01-07,2055,"Issaquah, Washington",Flying Tiger Line,841,Seattle - Cheyenne - Fort Jackson,Douglas C-54B-10-DO,N86574,18350,"{'total': 7, 'passengers': 3, 'crew': 4}","{'total': 7, 'passengers': 3, 'crew': 4}",0.0,The plane struck a tree on a mountain ridge at...,7,7,Flying Tiger Line,"{'place_id': 383391892, 'licence': 'Data © Ope...","Washington, United States"
2,1950,1953-01-07,0412,"Fish Haven, Idaho",Associated Air Transport,1-6-6A,Boeing Field - Cheyenne,Curtiss C-46,N1648M,22395,"{'total': 40, 'passengers': 37, 'crew': 3}","{'total': 40, 'passengers': 37, 'crew': 3}",0.0,The plane crashed into mountains while en rout...,40,40,Associated Air Transport,"{'place_id': 315821592, 'licence': 'Data © Ope...","Idaho, United States"


In [ ]:
def get_code(x):
    try:
        return df_country_codes_.loc[x]["alpha-3"]
    except:
        return float("nan")

In [ ]:
df_["Code"] = df_.Country_Geopy.apply(get_code)

/var/folders/sr/32z6khr97xs2g11w52pbkzww0000gn/T/ipykernel_49440/3812215558.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_["Code"] = df_.Country_Geopy.apply(get_code)


In [ ]:
df_["Code"].info()

<class 'pandas.core.series.Series'>
Index: 4918 entries, 0 to 5036
Series name: Code
Non-Null Count  Dtype 
--------------  ----- 
1641 non-null   object
dtypes: object(1)
memory usage: 205.9+ KB


In [ ]:
good = df_[df_.Code.apply(lambda x: not isinstance(x,float))]
good[:3]

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Country_Geopy,Code
3,1950,1953-01-15,0448,"25 nm off Agrigento, Italy",Military -Royal Air Force,-,None,Vickers Valetta Mk1 / Avero Lancaster,VX562 / TX270,None,"{'total': 26, 'passengers': 16, 'crew': 10}","{'total': 26, 'passengers': 16, 'crew': 10}",0.0,Both aircraft crashed after a midair collision...,26,26,Military,"{'place_id': 100666283, 'licence': 'Data © Ope...",Italy,ITA
12,1950,1953-03-14,None,"Kalasahar, India",Orient Airways,None,Delhi - Dacca,Convair CV-240-7,AP-AEG,082,"{'total': 16, 'passengers': 11, 'crew': 5}","{'total': 16, 'passengers': 11, 'crew': 5}",0.0,Crashed into a mountain while en route in seve...,16,16,Orient Airways,"{'place_id': 246884012, 'licence': 'Data © Ope...",India,IND
13,1950,1953-03-17,None,"Off Da Nang, Vietnam",Aigle Azur,None,Hue - Da Nang,Douglas DC-3,F-BEFG,19105,"{'total': 8, 'passengers': None, 'crew': None}","{'total': 8, 'passengers': None, 'crew': None}",0.0,Crashed into the sea in poor weather conditions.,8,8,Aigle Azur,"{'place_id': 253747752, 'licence': 'Data © Ope...",Vietnam,VNM


In [ ]:
{i for i in good.Country_Geopy if "United States" in i}

{'American Samoa, United States',
 'Guam, United States',
 'New York, United States',
 'Puerto Rico, United States',
 'Texas, United States',
 'United States',
 'United States Virgin Islands, United States'}

In [ ]:
df_country_codes_

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy
Country_Geopy,,,,,,,,,,,,
Afghanistan,Afghanistan,AF,AFG,4,ISO 3166-2:AF,Asia,Southern Asia,None,142.0,34.0,NaN,"{'place_id': 220100631, 'licence': 'Data © Ope..."
"Åland Islands, Finland",Åland Islands,AX,ALA,248,ISO 3166-2:AX,Europe,Northern Europe,None,150.0,154.0,NaN,"{'place_id': 180167715, 'licence': 'Data © Ope..."
Albania,Albania,AL,ALB,8,ISO 3166-2:AL,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 83081284, 'licence': 'Data © Open..."
Algeria,Algeria,DZ,DZA,12,ISO 3166-2:DZ,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 77146891, 'licence': 'Data © Open..."
"American Samoa, United States",American Samoa,AS,ASM,16,ISO 3166-2:AS,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 32805879, 'licence': 'Data © Open..."
...,...,...,...,...,...,...,...,...,...,...,...,...
"Wallis and Futuna, France",Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF,Oceania,Polynesia,None,9.0,61.0,NaN,"{'place_id': 33145748, 'licence': 'Data © Open..."
"Dakhla-Oued Ed-Dahab, Morocco",Western Sahara,EH,ESH,732,ISO 3166-2:EH,Africa,Northern Africa,None,2.0,15.0,NaN,"{'place_id': 288513039, 'licence': 'Data © Ope..."
Yemen,Yemen,YE,YEM,887,ISO 3166-2:YE,Asia,Western Asia,None,142.0,145.0,NaN,"{'place_id': 69324426, 'licence': 'Data © Open..."


In [ ]:
print(tabulate(df_country_codes_, headers='keys', tablefmt='psql'))

NameError: name 'tabulate' is not defined

In [ ]:
df_country_codes_[df_country_codes_["alpha-3"] == "VAT"]

,name,alpha-2,alpha-3,country-code,iso_3166-2,region,sub-region,intermediate-region,region-code,sub-region-code,intermediate-region-code,name_geopy
Country_Geopy,,,,,,,,,,,,
"Texas, United States",Holy See,VA,VAT,336,ISO 3166-2:VA,Europe,Southern Europe,None,150.0,39.0,NaN,"{'place_id': 293797351, 'licence': 'Data © Ope..."


In [ ]:
df_country_codes_.loc["Texas, United States"].name_geopy

{'place_id': 293797351,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 15440262,
 'lat': '30.0341947',
 'lon': '-95.8116685',
 'class': 'highway',
 'type': 'residential',
 'place_rank': 26,
 'importance': 0.10001,
 'addresstype': 'road',
 'name': 'Holy See',
 'display_name': 'Holy See, Harris County, Texas, 77447, United States',
 'boundingbox': ['30.0327048', '30.0355125', '-95.8116703', '-95.8114770']}

In [ ]:
good[good.Country_Geopy == 'Texas, United States']

,Decade,Date,Time,Location,Operator,Flight_Num,Route,Ac_Type,Registration,Cn_Ln,Aboard,Fatalities,Ground,Summary,Aboard_Total,Fatalities_Total,Operator_vs_Military,Country_location,Country_Geopy,Code
30,1950,1953-05-17,1415,"Near Marshall, Texas",Delta Air Lines,318,Dallas - Shreveport,Douglas DC-3,N28345,2224,"{'total': 20, 'passengers': 17, 'crew': 3}","{'total': 19, 'passengers': 16, 'crew': 3}",0.0,The plane crashed 13 miles east of Marashall T...,20,19,Delta Air Lines,"{'place_id': 305234088, 'licence': 'Data © Ope...","Texas, United States",VAT
127,1940,1947-10-08,0755,"Near El Paso, Texas",American Airlines,311,New York City - Dallas - Los Angeles,Douglas DC-4,NC90432,10314/DC45,"{'total': 54, 'passengers': 49, 'crew': 5}","{'total': 0, 'passengers': 0, 'crew': 0}",0.0,"While cruising at 8,000 ft. in clear weather, ...",54,0,American Airlines,"{'place_id': 305234088, 'licence': 'Data © Ope...","Texas, United States",VAT
276,1980,1985-03-12,None,"Fort Hood, Texas",Military - U.S. Air Force,None,Training,Lockheed C-130E Hercules,64-0549,4044,"{'total': 6, 'passengers': 0, 'crew': 6}","{'total': 6, 'passengers': 0, 'crew': 6}",0.0,Stalled during a training exercise and crashed.,6,6,Military,"{'place_id': 305209333, 'licence': 'Data © Ope...","Texas, United States",VAT
292,1980,1985-08-02,1806,"Ft. Worth-Dallas, Texas",Delta Air Lines,191,Fort Lauderdale - Dallas-Fort Worth,Lockheed L-1011-1 TriStar,N726DA,1163,"{'total': 163, 'passengers': 152, 'crew': 11}","{'total': 134, 'passengers': 126, 'crew': 8}",1.0,"While on a flight from Fort Lauderdale, FL to ...",163,134,Delta Air Lines,"{'place_id': 320023213, 'licence': 'Data © Ope...","Texas, United States",VAT
316,1980,1985-12-31,1714,"De Kalb, Texas",Private,None,Guntersville - Dallas/Ft. Worth,Douglas DC-3,N711Y,13658,"{'total': 9, 'passengers': 7, 'crew': 2}","{'total': 7, 'passengers': 7, 'crew': 0}",0.0,The plane crash landed into a field and struck...,9,7,Private,"{'place_id': 321005060, 'licence': 'Data © Ope...","Texas, United States",VAT
361,1990,1991-09-11,1003,"Near Eagle Lake, Texas",Continental Express,2574,Laredo - Houston,Embraer 120RT Brasilia,N33701,120-077,"{'total': 14, 'passengers': 11, 'crew': 3}","{'total': 14, 'passengers': 11, 'crew': 3}",0.0,In-flight loss of the leading edge of the left...,14,14,Continental Express,"{'place_id': 305234088, 'licence': 'Data © Ope...","Texas, United States",VAT
487,1950,1952-03-12,None,"Near Sequin, Texas",Military - U.S. Air Force / U.S. Air Force,None,Training,Boeing B-29 / Boeing B-29,None,None,"{'total': 15, 'passengers': 0, 'crew': 15}","{'total': 15, 'passengers': 0, 'crew': 15}",NaN,While on a training mission and flying blind o...,15,15,Military,"{'place_id': 305234088, 'licence': 'Data © Ope...","Texas, United States",VAT
552,1970,1978-04-04,2320,"Off Galveston, Texas",Air Logistics,None,None,Bell 212,N140AL,None,"{'total': 6, 'passengers': 4, 'crew': 2}","{'total': 3, 'passengers': 3, 'crew': 0}",0.0,The helicopter was attempting to land on an oi...,6,3,Air Logistics,"{'place_id': 294114023, 'licence': 'Data © Ope...","Texas, United States",VAT
930,1980,1986-10-04,0410,"San Antonio, Texas",Southern Air Transport,15,Kelly AFB - Warner Robins AFB,Lockheed L-100-30 Hercules,N15ST,4391,"{'total': 3, 'passengers': 0, 'crew': 3}","{'total': 3, 'passengers': 0, 'crew': 3}",0.0,The cargo plane lost altitude and crashed inve...,3,3,Southern Air Transport,"{'place_id': 304608347, 'licence': 'Data © Ope...","Texas, United States",VAT
973,1950,1951-06-03,1345,"New Boston, Texas",Military - U.S. Air Force,None,Fort Campbell - Lawton,Fairchild C-82,45-57761,10131,"{'total': 10, 'passengers': 0, 'crew': 10}","{'total': 10, 'passengers': 0, 'crew': 10}",0.0,Struck by lightning and crashed.,10,10,Military,"{'place_id': 320980508, 'licence': 'Data © Ope...","Texas, United States",VAT


In [ ]:
good.Country_Geopy.value_counts()

Country_Geopy
Russia                     138
Colombia                    76
Brazil                      66
New York, United States     59
Texas, United States        50
                          ... 
New Caledonia, France        1
Gibraltar                    1
Bhutan                       1
United States                1
Lesotho                      1
Name: count, Length: 160, dtype: int64

In [ ]:
bad = df_[df_.Code.apply(lambda x: isinstance(x,float))]

In [ ]:
k.Country_Geopy

NameError: name 'k' is not defined

In [ ]:
p = [i for i in k.Country_Geopy if "United States" in i]

In [ ]:
len(p)

926

In [ ]:
set(p)

{'Alabama, United States',
 'Alaska, United States',
 'Arizona, United States',
 'Arkansas, United States',
 'California, United States',
 'Colorado, United States',
 'Connecticut, United States',
 'Delaware, United States',
 'District of Columbia, United States',
 'Florida, United States',
 'Georgia, United States',
 'Hawaii, United States',
 'Idaho, United States',
 'Illinois, United States',
 'Indiana, United States',
 'Iowa, United States',
 'Kansas, United States',
 'Kentucky, United States',
 'Louisiana, United States',
 'Maine, United States',
 'Maryland, United States',
 'Massachusetts, United States',
 'Michigan, United States',
 'Minnesota, United States',
 'Mississippi, United States',
 'Missouri, United States',
 'Montana, United States',
 'Nebraska, United States',
 'Nevada, United States',
 'New Hampshire, United States',
 'New Jersey, United States',
 'New Mexico, United States',
 'North Carolina, United States',
 'Ohio, United States',
 'Oklahoma, United States',
 'Oreg

In [ ]:
# import json
# import pandas as pd
# import plotly.express as px

# with open("data/geojson-counties-fips.json", "r") as f:
#     counties = json.load(f)

# import pandas as pd
# df = pd.read_csv("data/fips-unemp-16.csv",
#                    dtype={"fips": str})

# fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
#                            color_continuous_scale="viridis",
#                            range_color=(0, 12),
#                         #    scope="usa",
#                            labels={'unemp':'unemployment rate'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
# import plotly.express as px

# # Sample data: a list of dictionaries
# # Each dictionary represents a point with a latitude, longitude, and some value.
# data = [
#     {"lat": 34.0522, "lon": -118.2437, "value": 10},  # Los Angeles
#     {"lat": 40.7128, "lon": -74.0060, "value": 20},  # New York
#     {"lat": 37.7749, "lon": -122.4194, "value": 30},  # San Francisco
# ]

# # Convert the list of dictionaries to a DataFrame
# import pandas as pd
# df = pd.DataFrame(data)

# # Plot using scatter_geo
# fig = px.scatter_geo(df,
#                      lat="lat",
#                      lon="lon",
#                      size="value",  # Adjust the size of each point based on the "value"
#                      )

# # Show the figure
# fig.show()


In [ ]:
# import plotly.express as px

# fig = px.choropleth(locations=["Northern Ireland, United Kingdom"], color=[1,2,3],)
# fig.show()

In [ ]:
# import plotly.express as px

# # Assuming 'df' is your DataFrame with 'lat' and 'lon' columns
# px.set_mapbox_access_token('your_mapbox_access_token_here')

# fig = px.density_mapbox(df, lat='lat', lon='lon', z='value', radius=10,
#                         center=dict(lat=0, lon=180), zoom=0,
#                         mapbox_style="stamen-terrain")

# fig.show()


In [ ]:
# import plotly.express as px
# import pandas as pd

# df = pd.read_csv("location_coordinate.csv")

# fig = px.scatter_geo(df,lat='lat',lon='long', hover_name="id")
# fig.update_layout(title = 'World map', title_x=0.5)
# fig.show()

In [ ]:
# df.Date = df.Date.astype(str)

In [ ]:
# df.to_json("data/silver/1920_2024_geopy.json")